In [1]:
from glob import glob
from itertools import compress
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage.io import imread,imshow,imsave 
from skimage.measure import regionprops_table
import time
import warnings
warnings.filterwarnings(action='ignore')
Image.MAX_IMAGE_PIXELS = None

os.chdir ('/Users/kim/Data/CRUK-OHSU_onedrive final 2022-09-30/Manuscript/IMC-IF method paper/Cell_report_method_resource/github')
 
print('library import done')

/Users/kim/opt/anaconda3/envs/scimap_env4/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


library import done


### Feature extraction from cell mask (for mean intensity )

In [2]:

def feature_extraction(name,mask): 

    s = regionprops_table(mask, intensity_image=mask, 
               properties=('label','mean_intensity','area','centroid','eccentricity','axis_major_length','axis_minor_length' ))
     
    df = pd.DataFrame(s)
    cell_id = df['mean_intensity']
    area = df['area']
    centroid_X = df['centroid-1'] 
    centroid_Y = df['centroid-0']
    eccentricity = df['eccentricity']
    axis_major_length = df['axis_major_length']
    axis_minor_length = df['axis_minor_length']

    df = pd.concat([cell_id, area, centroid_X, centroid_Y, eccentricity, axis_major_length, axis_minor_length], axis=1, join='outer')
    df.rename(columns={'mean_intensity':'cell_id'}, inplace=True)
    df.rename(columns={'centroid-1':'centroid_X'}, inplace=True)
    df.rename(columns={'centroid-0':'centroid_Y'}, inplace=True)
    
    # IMC regionprop 
    img_fn = f'Analysis/new_July_2021/IMC_Registration/*{name}*/*ome.tiff'
    marker_list = sorted(glob(img_fn))[8:-4]  
    marker_name_list=[m.split('/')[-1].split('_')[-1].replace('.ome.tiff','') for m in marker_list] 
    
    i = 0        
    for i in range(len(marker_list)):
        marker=imread(marker_list[i])
        table = regionprops_table(mask, intensity_image = marker, properties=('label','mean_intensity','centroid'))
        df_table = pd.DataFrame(table)
        df_total = pd.concat([df,df_table],axis=1,join='outer')
        markername=marker_name_list[i]
        df_total.rename(columns={'mean_intensity': markername},inplace=True)
        df=df_total
        i=i+1 
        print(markername)
    print(name, 'done')
    return df
print('done')

done


In [3]:
#1. IF_cell
#2. IF_nuc_final
#3. IMC_cell 
#4. IMC_nuc_final 

### Feature extraction from nuclear mask (for nucleus area calculation)

In [4]:
def feature_extraction_nuc(name,mask): 

    s = regionprops_table(mask, intensity_image=mask, 
               properties=('label','mean_intensity','area','centroid','eccentricity' ))
     
    df = pd.DataFrame(s)
    cell_id = df['mean_intensity']
    area = df['area']
    centroid_X = df['centroid-1'] 
    centroid_Y = df['centroid-0']
    eccentricity = df['eccentricity']

    df = pd.concat([cell_id, area, centroid_X, centroid_Y, eccentricity], axis=1, join='outer')
    df.rename(columns={'mean_intensity':'cell_id'}, inplace=True)
    df.rename(columns={'centroid-1':'centroid_X'}, inplace=True)
    df.rename(columns={'centroid-0':'centroid_Y'}, inplace=True)
    df.rename(columns={'area':'area_nuc'}, inplace=True)

    return df
print('done')

done


In [5]:
def feature(name): 

    pt=name.split('.')[0]

    # IMC images 
    IMC_DNA = imread(glob(f'Analysis/new_July_2021/IMC_Registration/*{name}*/*191_DNA*')[0])
    IMC_Ecad = imread(glob(f'Analysis/new_July_2021/IMC_Registration/*{name}*/*E-Ca*')[0])

    # registered images    
    rCD45 = imread(glob(f'Data/IMC EMR (1)/*{name}*/resized_IF/*{name}*_CD45.tif')[0]) 
    rDAPI = imread(glob(f'Data/IMC EMR (1)/*{name}*/resized_IF/*{name}*_DAPI.tif')[0]) 
    rEcad = imread(glob(f'Data/IMC EMR (1)/*{name}*/resized_IF/*{name}*_Ecad.tif')[0]) 

    # masks 
    IF_cell = imread(f'Analysis/new_July_2021/maskmerge_mesmer_0325/{name}.tiff') 
    IF_nuc = imread(f'Analysis/new_July_2021/maskmerge_mesmer_nuc_0325/{name}.tiff')     
    IMC_cell = imread(f'Analysis/new_July_2021/IMC_mesmer/{pt}.tiff') 
    IMC_nuc = imread(f'Analysis/new_July_2021/IMC_mesmer_nuc/{pt}.tiff')

    IMC_rim = np.where(IMC_nuc != 0, 0, IMC_cell)
    IF_rim = np.where(IF_nuc != 0, 0, IF_cell)

    IF_nuc_final = np.where(IF_rim !=0, 0, IF_cell)
    IMC_nuc_final = np.where(IMC_rim !=0, 0, IMC_cell)

    # IF 
    IF_cell = feature_extraction (name= name, mask = IF_cell)
    IF_nuc= feature_extraction_nuc(name=name, mask= IF_nuc_final)

    img_fn = f'Analysis/new_July_2021/IMC_Registration/*{name}*/*ome.tiff'
    marker_list = sorted(glob(img_fn))[8:-4]  
    col1=[m.split('/')[-1].split('_')[-1].replace('.ome.tiff','') for m in marker_list] 
    col2 = ['cell_id', 'area', 'centroid_X','centroid_Y', 'eccentricity','axis_major_length','axis_minor_length']
    col=col2+col1
    IF_cell_final = IF_cell[col]

    IF_total= pd.merge(IF_cell_final, IF_nuc, on = ['cell_id'], how='inner')
    IF_total.rename(columns={'centroid_X_x':'centroid_X'}, inplace=True)
    IF_total.rename(columns={'centroid_Y_x':'centroid_Y'}, inplace=True)

    IF_total = IF_total.drop(columns = ['centroid_X_y','centroid_Y_y'])
    IF_total.rename(columns={'eccentricity_x':'eccentricity_cell'}, inplace=True)
    IF_total.rename(columns={'eccentricity_y':'eccentricity_nuc'}, inplace=True)
    IF_total.to_csv(f'Analysis/new_July_2021/ncratio_feature/IF_{name}.csv') 
    print('done')

    # IMC 
    IMC_cell = feature_extraction (name= name, mask = IMC_cell)
    IMC_nuc= feature_extraction_nuc(name=name, mask= IMC_nuc_final)

    img_fn = f'Analysis/new_July_2021/IMC_Registration/*{name}*/*ome.tiff'
    marker_list = sorted(glob(img_fn))[8:-4]  
    col1=[m.split('/')[-1].split('_')[-1].replace('.ome.tiff','') for m in marker_list] 
    col2 = ['cell_id', 'area', 'centroid_X','centroid_Y', 'eccentricity','axis_major_length','axis_minor_length']
    col=col2+col1
    IMC_cell_final = IMC_cell[col]

    IMC_total= pd.merge(IMC_cell_final, IMC_nuc, on = ['cell_id'], how='inner') # Matching cell and nuleus ID 
    IMC_total.rename(columns={'centroid_X_x':'centroid_X'}, inplace=True)
    IMC_total.rename(columns={'centroid_Y_x':'centroid_Y'}, inplace=True)

    IMC_total = IMC_total.drop(columns = ['centroid_X_y','centroid_Y_y'])
    IMC_total.rename(columns={'eccentricity_x':'eccentricity_cell'}, inplace=True)
    IMC_total.rename(columns={'eccentricity_y':'eccentricity_nuc'}, inplace=True)
    IMC_total.to_csv(f'Analysis/new_July_2021/ncratio_feature/IMC_{name}.csv') 
    
    print('done')
print('done')

done


In [6]:
%time
names = ['DE18.NE', 'DE28.NE.A', 'PS14.24197-1E', 'PS15.19650-B3', 'PS16.38634-F2']

for name in names:
    feature(name=name)
print('finally done')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
aSMA
CD31
p53
CD11b
CD38
CD16
CD163
panKt
CCR3
PDL1
LAG3
CD1a
CD206
CSF1R
Foxp3
CD4
E-Cad
CD68
ICOS
CD20
CD8a
HLA-DR
PD1
VISTA
CTLA4
GranB
Ki67
Col1
CD3
Tryptase
CCR7
CD45RO
TIM3
CD56
CD11c
DE18.NE done
done
aSMA
CD31
p53
CD11b
CD38
CD16
CD163
panKt
CCR3
PDL1
LAG3
CD1a
CD206
CSF1R
Foxp3
CD4
E-Cad
CD68
ICOS
CD20
CD8a
HLA-DR
PD1
VISTA
CTLA4
GranB
Ki67
Col1
CD3
Tryptase
CCR7
CD45RO
TIM3
CD56
CD11c
DE18.NE done
done
aSMA
CD31
p53
CD11b
CD38
CD16
CD163
panKt
CCR3
PDL1
LAG3
CD1a
CD206
CSF1R
Foxp3
CD4
E-Cad
CD68
ICOS
CD20
CD8a
HLA-DR
PD1
VISTA
CTLA4
GranB
Ki67
Col1
CD3
Tryptase
CCR7
CD45RO
TIM3
CD56
CD11c
DE28.NE.A done
done
aSMA
CD31
p53
CD11b
CD38
CD16
CD163
panKt
CCR3
PDL1
LAG3
CD1a
CD206
CSF1R
Foxp3
CD4
E-Cad
CD68
ICOS
CD20
CD8a
HLA-DR
PD1
VISTA
CTLA4
GranB
Ki67
Col1
CD3
Tryptase
CCR7
CD45RO
TIM3
CD56
CD11c
DE28.NE.A done
done
aSMA
CD31
p53
CD11b
CD38
CD16
CD163
panKt
CCR3
PDL1
LAG3
CD1a
CD206
CSF1R
Foxp3
CD4
E-Cad
CD68
ICOS


### Feature table merge

In [ ]:
# Feature merge IF

file_list= sorted(glob('Analysis/new_July_2021/ncratio_feature/IF_*'))

df_mask_column =pd.read_csv(file_list[0])
column_names=df_mask_column.columns
df_merg = pd.DataFrame(columns = column_names)

i=0
for i in range(len(file_list)):
    name= file_list[i].split('/')[-1].split('_')[-1].replace('.csv','')
    df_next = pd.read_csv(file_list[i])
    df_next['name']= name
    df_merg = pd.concat([df_merg,df_next], ignore_index=True)
    i=i+1   

df_merg.rename(columns={'area':'area_cell'}, inplace=True)
df_merg['area_rim'] = df_merg['area_cell'] - df_merg['area_nuc']
df_merg.to_csv('Analysis/new_July_2021/ncratio_feature/merge_IF.csv')
print('done')

In [ ]:
# Feature merge IMC 

file_list= sorted(glob('Analysis/new_July_2021/ncratio_feature/IMC_*'))

df_mask_column =pd.read_csv(file_list[0])
column_names=df_mask_column.columns
df_merg = pd.DataFrame(columns = column_names)

i=0
for i in range(len(file_list)):
    name= file_list[i].split('/')[-1].split('_')[-1].replace('.csv','')
    df_next = pd.read_csv(file_list[i])
    df_next['name']= name
    df_merg = pd.concat([df_merg,df_next], ignore_index=True)
    i=i+1   

df_merg.rename(columns={'area':'area_cell'}, inplace=True)
df_merg['area_rim'] = df_merg['area_cell'] - df_merg['area_nuc']
df_merg.to_csv('Analysis/new_July_2021/ncratio_feature/merge_IMC.csv')
print('done')